In [46]:
import sys
sys.path.append('../Custom Library/') 
import datetime as dt
import HR_Analytics as hr
import pandas as pd

In [47]:
engine = hr.setEngine('credentials','HRODS')

In [48]:
queryActual = """
Select
    Extract(Year From D."Report Date") As "Year",
    get_f_year_quarter('Q', D."Report Date")  As "Quarter",
    D."Metric",
    Sum(D."Headcount") As "BAA Headcount"
From (
    Select
        "Report Date", 
        'B/AA Technical' As "Metric",
        1 As "Headcount",
        'Exempt' As "Table"
    From 
        "HPW_DATA" As  WD
    Left Join
        "TECHNICAL_JOBS" AS TT ON TT."Job Family Code" = WD."Job Family Code"
    Where 
        "Worker Reg / Temp Code" = 'R'
    And 
        "Worker Status Category Code" = 'A'
    And
        "Pay Group Country Desc" = 'United States of America'
    And
        "Ethnicity Group" = 'BLACK_USA'
    And
        Case
            When TT."Job Family Code" Is Null Then 'No'
            Else 'Yes'
        End = 'Yes'
    And
        "Exempt" = 'Y'
        
    Union All

    Select 
        "Report Date",
        'B/AA VP+ level' As "Metric",
        1 As "Headcount",
        'Exempt' As "Table"
    From
        "HPW_DATA" As  WD
    Left Join
        "TECHNICAL_JOBS" AS TT ON TT."Job Family Code" = WD."Job Family Code"
    Where 
        "Worker Reg / Temp Code" = 'R'
    And 
        "Worker Status Category Code" = 'A'
    And
        "Pay Group Country Desc" = 'United States of America'
    And
        "Ethnicity Group" = 'BLACK_USA'
    And 
        "Management Level" In ('EXE','SFL','FEL')
    And
            "Exempt" = 'Y'
) As D
Where 
    D."Report Date" In (
        '2020-01-31',
        '2020-04-30',
        '2020-07-31',
        '2020-10-31',
        '2021-01-31',
        '2021-04-30',
        '2021-07-31',
        '2021-10-31'
    )

Group By 
    D."Report Date",
    D."Table",
    D."Metric"
"""

actualRep = hr.internalQuery(queryActual, engine)

In [90]:
queryPopulation = """
Select
    Extract(Year From D."Report Date") As "Year",
    get_f_year_quarter('Q', D."Report Date")  As "Quarter",
    D."Metric",
    Sum(D."Headcount") As "Headcount"
From (
    Select
        "Report Date", 
        'B/AA Technical' As "Metric",
        1 As "Headcount",
        'Exempt' As "Table"
    From 
        "HPW_DATA" As  WD
    Left Join
        "TECHNICAL_JOBS" AS TT ON TT."Job Family Code" = WD."Job Family Code"
    Where 
        "Worker Reg / Temp Code" = 'R'
    And 
        "Worker Status Category Code" = 'A'
    And
        "Pay Group Country Desc" = 'United States of America'
    And
        Case
            When TT."Job Family Code" Is Null Then 'No'
            Else 'Yes'
        End = 'Yes'
    And
        "Exempt" = 'Y'
        
    Union All

    Select 
        "Report Date",
        'B/AA VP+ level' As "Metric",
        1 As "Headcount",
        'Exempt' As "Table"
    From
        "HPW_DATA" As  WD
    Left Join
        "TECHNICAL_JOBS" AS TT ON TT."Job Family Code" = WD."Job Family Code"
    Where 
        "Worker Reg / Temp Code" = 'R'
    And 
        "Worker Status Category Code" = 'A'
    And
        "Pay Group Country Desc" = 'United States of America'
    And 
        "Management Level" In ('EXE','SFL','FEL')
    And
            "Exempt" = 'Y'
) As D
Where 
    D."Report Date" In (
        '2020-01-31',
        '2020-04-30',
        '2020-07-31',
        '2020-10-31',
        '2021-01-31',
        '2021-04-30',
        '2021-07-31',
        '2021-10-31'
    )

Group By 
    D."Report Date",
    D."Table",
    D."Metric"
"""
actualPop = hr.internalQuery(queryPopulation, engine)

In [96]:
data = pd.merge(actualRep,actualPop, on=['Year','Quarter','Metric'])
data['%'] = round((data['BAA Headcount'] / data['Headcount']) * 100,1)
del data['Headcount']

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')

In [95]:
data.pivot(index=['Year','Quarter'], columns='Metric', values=['BAA Headcount','%'])



BAA Headcount                             %               
Metric         B/AA Technical B/AA VP+ level B/AA Technical B/AA VP+ level
Year   Quarter                                                            
2020.0 Q1                91.0            7.0            2.0            3.2
       Q2                90.0            6.0            2.1            2.8
       Q3                94.0            6.0            2.3            2.9
       Q4                91.0            8.0            2.3            4.0
2021.0 Q1                94.0            9.0            2.4            4.6
       Q2                95.0            9.0            2.4            4.4
       Q3               103.0            8.0            2.6            4.1